In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from utils.objects import *
from utils.main_objects import *

## Importing the data

In [2]:
rsu = pd.read_csv('data/rsu.csv')
es = pd.read_csv('data/es.csv')
tasks = pd.read_csv('data/tasks.csv')

In [3]:
rsu_bank = populate_rsus(rsu)
es_bank = populate_ess(es)
tasks_bank = populate_tasks(tasks)

In [4]:
len(rsu_bank), len(es_bank), len(tasks_bank)

(100, 100, 50)

## Generating a network

In [5]:
# we repopulate the rsu and es banks to ensure the network is new
rsu_bank = populate_rsus(rsu)
es_bank = populate_ess(es)
network = get_network(rsu_bank, es_bank)

In [6]:
# put the network in a dataframe with the attributes of the objects as columns
# network is a list of objects
network_df = pd.DataFrame([vars(x) for x in network])
network_df.head()

,ID,X,Y,DTR,STATE,END_TIME,ES
0,2,7,71,5,IDLE,None,<utils.objects.ES object at 0x11ea36680>
1,16,0,1,24,IDLE,None,<utils.objects.ES object at 0x11e9dbe50>
2,11,41,61,48,IDLE,None,<utils.objects.ES object at 0x11ea20610>
3,18,93,75,12,IDLE,None,AP
4,76,92,17,30,IDLE,None,<utils.objects.ES object at 0x11e9dbaf0>


In [7]:
tasks_df = pd.DataFrame([vars(x) for x in tasks_bank])
tasks_df.head()

,ID,LENGTH,FILE_SIZE,TYPE,X,Y,COMPLETED,MIGRATION_HISTORY,COMPUTATION_HISTORY,MIGRATION_TIME
0,0,12,322,DATA TRANSFER,73,32,False,0,0,0
1,1,23,483,DATA TRANSFER,83,21,False,0,0,0
2,2,83,103,DATA TRANSFER,49,65,False,0,0,0
3,3,74,3,COMPUTATION,24,50,False,0,0,0
4,4,76,467,COMPUTATION,23,52,False,0,0,0


## Running the task offloading process

In [9]:
current_time = 0

# while there are still tasks to be completed
while not all(task.COMPLETED for task in tasks_bank):

    # get the first task in the task bank which is not completed
    task = next((task for task in tasks_bank if task.COMPLETED == False), None)

    # get the closest rsu to the task
    closest_rsu = get_closest_rsu(rsu_bank, task)

    # suppose the task is assigned at that rsu directly
    task.X = closest_rsu.X
    task.Y = closest_rsu.Y

    # if the rsu is free and compatible with the task, compute the task
    if is_server_free(closest_rsu) and compatible(closest_rsu, task):
        closest_rsu.STATE = "BUSY"
        task.COMPLETED = True
        # if the task is of type 'COMPUTATION', compute the computation time
        if task.TYPE == "COMPUTATION":
            t_computation = computation_time(task_length=task.LENGTH,
                                                vm_nb=closest_rsu.VM_NB,
                                                vm_cpu=closest_rsu.VM_CP)
            closest_rsu.END_TIME = current_time + t_computation
            task.COMPUTATION_HISTORY = t_computation
        # else (if the task is of type 'DATA TRANSFER'), compute the transfer time
        elif task.TYPE == "DATA TRANSFER":
            t_data_transfer = data_transfer_time(file_size=task.FILE_SIZE,
                                          dtr=closest_rsu.DTR)
            closest_rsu.END_TIME = current_time + t_data_transfer
            task.COMPUTATION_HISTORY = t_data_transfer

    # if the rsu is busy or incompatible with the task, migrate the task
    else:
        # find the closest rsu to the task
        next_closest_rsu = get_closest_rsu(rsu_bank, task)
        
        # compute the migration time
        t_migration = migration_time(file_size=task.FILE_SIZE,
                                        distance=distance(task, next_closest_rsu),
                                        dtr=closest_rsu.DTR)
        
        # update the migration time of the task
        task.MIGRATION_TIME = t_migration
        # keep a record of the total migration time of the task
        task.MIGRATION_HISTORY += t_migration

        # update the coordinates of the task for when it will be migrated
        task.X = next_closest_rsu.X
        task.Y = next_closest_rsu.Y

    # check if any rsu should turn free at the current time
    for rsu in rsu_bank:
        if rsu.STATE == "BUSY" and rsu.END_TIME == current_time:
                rsu.STATE = "IDLE"
                rsu.END_TIME = None

    # remove a time unit from the migration time of each task if the migration time is greater than 0
    for task in tasks_bank:
        if task.MIGRATION_TIME > 0:
            task.MIGRATION_TIME -= 1

    # update the current time
    current_time += 1



In [ ]:
task.FILE_SIZE

322